# Similar Hood

### Data Analysis

First thing that we will need to do is to import the nessecary libraries.

In [50]:
import pandas as pd
import numpy as np
import json 
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

Next I'm going to sset my client key and secret for the api call i'm going to make

In [199]:
CLIENT_ID = 'XQPYR51KCT4ZEWGFAVLTWMLT2MYIQ04UGE1OFEPN3C0UBPTZ' # your Foursquare ID
CLIENT_SECRET = '1M3O3NX140P1Z32RVSA3XACC0DZOBAYJ4EAAIRC5HLJH4133' # your Foursquare Secret
VERSION = '20200205'
LIMIT = 50

Now we will proceede to build the data that will be used in this project, first I will use beautifulsoup to scrap the Wiki page that has all the relevant information. From the html that I get I will then import the values into a data frame.

In [11]:
def scrapper(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    results = soup.findAll("table", {"class": "wikitable sortable"})
    return str(results[0])
a = scrapper('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [13]:
#data cleaning
df = pd.read_html(a)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


As you can see the dataframe above has various miss in data points, so I am going to need to clean up the data a bit. I will do this by dropping the rows in which the value of the borough is "Not assinged". Then I will combine all the rows with the same postcode value and combine the neighbourhoods seprated by a comma

In [14]:
df = df.drop(df[df.Borough == 'Not assigned'].index)
agg_function = {'Borough':'first','Neighbourhood':', '.join}
df = df.groupby(df['Postcode']).agg(agg_function).reset_index()
df['Neighbourhood'] = df.apply(
    lambda row: row['Borough'] if row['Neighbourhood'] == 'Not assigned' else row['Neighbourhood'],axis=1)
df = df.rename(columns={'Postcode':'PostalCode'})
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now I will add the Longitude and Latitude for each postcode

In [18]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data ')
geo_df = geo_df.rename(columns={'Postal Code':'PostalCode'})
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
merge_df = pd.merge(left=df,right=geo_df,how='left', left_on='PostalCode',right_on='PostalCode')
merge_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Next we are going to pull the top 100 venus in a 1km radius using the Foursquare API

In [200]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [201]:
toronto_venues = getNearbyVenues(names=merge_df['PostalCode'],
                                   latitudes=merge_df['Latitude'],
                                   longitudes=merge_df['Longitude']
                                  )
toronto_venues.head()

,PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
1,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,M1B,43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
3,M1B,43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
4,M1B,43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop


This dataframe now contians the neighborhood the venue name annd the latitude and longitude that is associated with the venue and wat category it falls under. I'm going to create a new dataframe where the Neighborhood and venue Latitude/Longitude is dropped. For this project what we are look for most is the kinds of venues in each area. So our features will be the Postal Code of the area and the number of each kind of venue of each possible categories. To do this we will first need to extract the unique categories from the dataframe then sum up how many of each are in a particular postal code. 

[ PostalCode,Venue categoy 1,Venue categoy 2,....] and so on will be the columns of the data frame

In [202]:
data = toronto_venues.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue', 'Venue Latitude', 'Venue Longitude'], axis = 1)
unique = data['Venue Category'].nunique()
print('The number of unique venues are: ', unique)
data.head()

The number of unique venues are:  306


,PostalCode,Venue Category
0,M1B,Fast Food Restaurant
1,M1B,Fast Food Restaurant
2,M1B,Caribbean Restaurant
3,M1B,Paper / Office Supplies Store
4,M1B,Coffee Shop


This is a list of all the different venue categories in the dataframe

In [212]:
list_un = data['Venue Category'].unique().tolist()
unique_hoods = data['PostalCode'].unique().tolist()

Now that we know there are 306 venue categories the next step it to determine how many of those occure in each neighbourhood. To do this we first need to group by the neighbourhood then count the number of occurences of a particular type in a nieghborhood

In [209]:
final_df = data.groupby(['PostalCode'])['Venue Category'].value_counts().unstack(level=1).fillna(0)
final_df = final_df.rename_axis(None, axis=1).reset_index(drop=False)
final_df.head()


,PostalCode,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


### Feature Selection
    

Now that we the data set up in the way that we want, the next thing we are going to do is select the features that we are going to use in our clustering model.

In [213]:
X = final_df[list_un]
X[0:5]

,Fast Food Restaurant,Caribbean Restaurant,Paper / Office Supplies Store,Coffee Shop,Hobby Shop,Bus Station,Trail,African Restaurant,Auto Workshop,Chinese Restaurant,...,Supplement Shop,Tanning Salon,Kids Store,College Rec Center,Shopping Plaza,Transportation Service,Empanada Restaurant,ATM,Storage Facility,Golf Driving Range
0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Import the model and fit it to the data

In [214]:
from sklearn.cluster import KMeans

In [215]:
km = KMeans(n_clusters=3)
km.fit(X)
km.predict(X)
labels = km.labels_